In [10]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [11]:
import pandas as pd
import numpy as np

from memory_fix import reduce_mem_usage

In [12]:
transactions_train = reduce_mem_usage( pd.read_csv('data/transactions_train.csv') )

Memory usage of dataframe is 668.09 MB
Memory usage after optimization is: 250.53 MB
Decreased by 62.5%


In [13]:
group_text = pd.read_excel('data/small_group_description.xlsx')

In [14]:
from sklearn.cluster import SpectralClustering
import scipy.sparse as sps


In [15]:
def get_diff(array):
    new_array = list()
    for i in range(1, len(array)):
        new_array.append(array[i] - array[i-1])
    
    return np.mean(new_array), np.max(new_array), np.min(new_array), np.std(new_array)

In [16]:
from tqdm import tqdm

In [17]:
def get_target_stat(transaction, target):
    stat = {'mean_date':[], 'max_date':[],'min_date':[],'std_date':[], 'mean_sum':[], 'mean_count':[]}
    
    small = transaction[transactions_train.small_group == target]
    group = small.groupby(['client_dk', 'trans_date'])["amount"].agg(['count', 'sum'])
    
    for ind in tqdm(transaction['client_dk'].unique()):
        try:
            smb = group.loc[ind]
            mean, mx, mn, std = get_diff(list(smb.index))

            stat['mean_date'].append(mean)
            stat['max_date'].append(mx)
            stat['min_date'].append(mn)
            stat['std_date'].append(std)

            stat['mean_sum'].append(np.mean(smb['sum']))
            stat['mean_count'].append(np.mean(smb['count']))
        except Exception:
            stat['mean_date'].append(-10000)
            stat['max_date'].append(-10000)
            stat['min_date'].append(-10000)
            stat['std_date'].append(-10000)

            stat['mean_sum'].append(-10000)
            stat['mean_count'].append(-10000)
            
        
    return stat

In [18]:
ds = get_target_stat(transactions_train, target=27)

100%|██████████| 25000/25000 [00:07<00:00, 3290.47it/s]


In [21]:
ds = pd.DataFrame(ds)

In [22]:
ds.to_csv("27_features_train.csv")